In [11]:
import pandas as pd
import numpy as np
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
import string
from nltk import pos_tag
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import wordnet
from sklearn.model_selection import train_test_split
import nltk

In [14]:
f1 = r"D:\Python\Datasets\training_twitter_x_y_train.csv"
f2 = r"D:\Python\Datasets\test_twitter_x_test.csv"

In [15]:
df = pd.read_csv(f1)
df2 = pd.read_csv(f2)

In [16]:
df.head()

,tweet_id,airline_sentiment,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,567900433542488064,negative,Southwest,NaN,ColeyGirouard,NaN,0,"@SouthwestAir I am scheduled for the morning, ...",NaN,2015-02-17 20:16:29 -0800,Washington D.C.,Atlantic Time (Canada)
1,569989168903819264,positive,Southwest,NaN,WalterFaddoul,NaN,0,@SouthwestAir seeing your workers time in and ...,NaN,2015-02-23 14:36:22 -0800,"Indianapolis, Indiana; USA",Central Time (US & Canada)
2,568089179520954368,positive,United,NaN,LocalKyle,NaN,0,@united Flew ORD to Miami and back and had gr...,NaN,2015-02-18 08:46:29 -0800,Illinois,Central Time (US & Canada)
3,568928195581513728,negative,Southwest,NaN,amccarthy19,NaN,0,@SouthwestAir @dultch97 that's horse radish 😤🐴,NaN,2015-02-20 16:20:26 -0800,NaN,Atlantic Time (Canada)
4,568594180014014464,negative,United,NaN,J_Okayy,NaN,0,@united so our flight into ORD was delayed bec...,NaN,2015-02-19 18:13:11 -0800,NaN,Eastern Time (US & Canada)


In [17]:
y_train = df['airline_sentiment']
print(y_train)

0        negative
1        positive
2        positive
3        negative
4        negative
           ...   
10975     neutral
10976    positive
10977    negative
10978    negative
10979    negative
Name: airline_sentiment, Length: 10980, dtype: object


In [18]:
x_train = df.copy()
x_test = df2.copy()
x_train.head()

,tweet_id,airline_sentiment,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,567900433542488064,negative,Southwest,NaN,ColeyGirouard,NaN,0,"@SouthwestAir I am scheduled for the morning, ...",NaN,2015-02-17 20:16:29 -0800,Washington D.C.,Atlantic Time (Canada)
1,569989168903819264,positive,Southwest,NaN,WalterFaddoul,NaN,0,@SouthwestAir seeing your workers time in and ...,NaN,2015-02-23 14:36:22 -0800,"Indianapolis, Indiana; USA",Central Time (US & Canada)
2,568089179520954368,positive,United,NaN,LocalKyle,NaN,0,@united Flew ORD to Miami and back and had gr...,NaN,2015-02-18 08:46:29 -0800,Illinois,Central Time (US & Canada)
3,568928195581513728,negative,Southwest,NaN,amccarthy19,NaN,0,@SouthwestAir @dultch97 that's horse radish 😤🐴,NaN,2015-02-20 16:20:26 -0800,NaN,Atlantic Time (Canada)
4,568594180014014464,negative,United,NaN,J_Okayy,NaN,0,@united so our flight into ORD was delayed bec...,NaN,2015-02-19 18:13:11 -0800,NaN,Eastern Time (US & Canada)


In [8]:
x_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10980 entries, 0 to 10979
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   tweet_id                10980 non-null  int64 
 1   airline_sentiment       10980 non-null  object
 2   airline                 10980 non-null  object
 3   airline_sentiment_gold  31 non-null     object
 4   name                    10980 non-null  object
 5   negativereason_gold     24 non-null     object
 6   retweet_count           10980 non-null  int64 
 7   text                    10980 non-null  object
 8   tweet_coord             776 non-null    object
 9   tweet_created           10980 non-null  object
 10  tweet_location          7430 non-null   object
 11  user_timezone           7403 non-null   object
dtypes: int64(2), object(10)
memory usage: 1.0+ MB


In [20]:
text = df['text']
text2 = x_test['text']

In [12]:
def get_simple_pos(word):
    pos = pos_tag([word])[0][1]
    
    if pos.startswith('J'):
        return wordnet.ADJ
    elif pos.startswith('N'):
        return wordnet.NOUN
    elif pos.startswith('V'):
        return wordnet.VERB
    elif pos.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN
    
    
def clean_review(words):
    lemmatizer = WordNetLemmatizer()
    stop = set(stopwords.words('english'))
    punctuations = list(string.punctuation)
    stop.update(punctuations)
    output = []
    for word in words:
        if word.lower() not in stop:
            pos = pos_tag([word])
            clean_word = lemmatizer.lemmatize(word, pos = get_simple_pos(word))
            output.append(clean_word.lower())
            
    return output

def get_feature_dict(words):
    doc_dict = {}
    word_set = set(words)
    
    for w in features:
        doc_dict[w] = w in word_set
            
    return doc_dict

In [21]:
documents = []

for i in range(len(y_train)):
    documents.append((word_tokenize(text[i]),y_train[i]))
    
test_documents = []

for i in range(len(x_test)):
    test_documents.append(word_tokenize(text2[i]))

In [26]:
print(documents[:4])

[(['@', 'SouthwestAir', 'I', 'am', 'scheduled', 'for', 'the', 'morning', ',', '2', 'days', 'after', 'the', 'fact', ',', 'yes', '..', 'not', 'sure', 'why', 'my', 'evening', 'flight', 'was', 'the', 'only', 'one', 'Cancelled', 'Flightled'], 'negative'), (['@', 'SouthwestAir', 'seeing', 'your', 'workers', 'time', 'in', 'and', 'time', 'out', 'going', 'above', 'and', 'beyond', 'is', 'why', 'I', 'love', 'flying', 'with', 'you', 'guys', '.', 'Thank', 'you', '!'], 'positive'), (['@', 'united', 'Flew', 'ORD', 'to', 'Miami', 'and', 'back', 'and', 'had', 'great', 'crew', ',', 'service', 'on', 'both', 'legs', '.', 'THANKS'], 'positive'), (['@', 'SouthwestAir', '@', 'dultch97', 'that', "'s", 'horse', 'radish', '😤🐴'], 'negative')]


In [23]:
documents_cleaned = [(clean_review(doc),category) for doc, category in documents]
test_documents_cleaned = [clean_review(doc) for doc in test_documents]

In [24]:
training_documents = documents_cleaned
testing_documents = test_documents_cleaned

In [27]:
print(training_documents[:4])
print(testing_documents[:2])

[(['southwestair', 'schedule', 'morning', '2', 'day', 'fact', 'yes', '..', 'sure', 'even', 'flight', 'one', 'cancelled', 'flightled'], 'negative'), (['southwestair', 'see', 'worker', 'time', 'time', 'go', 'beyond', 'love', 'fly', 'guy', 'thank'], 'positive'), (['united', 'flew', 'ord', 'miami', 'back', 'great', 'crew', 'service', 'leg', 'thanks'], 'positive'), (['southwestair', 'dultch97', "'s", 'horse', 'radish', '😤🐴'], 'negative')]
[['americanair', 'car', 'gng', 'dfw', 'pulled', '1hr', 'ago', 'icy', 'road', 'on-hold', 'aa', 'since', '1hr', 'ca', "n't", 'reach', 'arpt', 'aa2450', 'wat', '2'], ['americanair', 'plane', '’', 'land', 'identical', 'bad', 'condition', 'grk', 'accord', 'metars']]
